In [2]:
from mysql.connector import MySQLConnection, Error

In [5]:
def make_connection():
    """
    Make a connection to the school database.
    """
    try:
        conn = MySQLConnection(host = 'IES-ADS-ClassDB.sjsu.edu',
                               database = 'querycrew_wh',
                               user = 'querycrew_user',
                               password = 'Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [7]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [9]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton,
    QLabel, QComboBox, QMessageBox, QTableView, QHeaderView
)
from PyQt5.QtSql import QSqlDatabase, QSqlTableModel, QSqlQuery, QSqlQueryModel

class OperationalApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Query Crew Main Menu")
        self.resize(800, 600)

        # Initialize the database connection
        self.db = self.init_db()
        self.init_menu()

    def init_db(self):
        db = QSqlDatabase.addDatabase("QMYSQL")
        db.setHostName("IES-ADS-ClassDB.sjsu.edu")
        db.setDatabaseName("querycrew_wh")
        db.setUserName("querycrew_user")
        db.setPassword("Pomegranate_746")
        if not db.open():
            QMessageBox.critical(None, "DB Connection Error", db.lastError().text())
            sys.exit(1)
        return db

    def init_menu(self):
        menu = QWidget()
        layout = QVBoxLayout(menu)

        layout.addWidget(QLabel("Welcome to Query Crew!"))
        layout.addWidget(QLabel("Select an action:"))

        btn1 = QPushButton("View Customers")
        btn1.clicked.connect(lambda: self.show_table("Customers"))
        layout.addWidget(btn1)

        btn2 = QPushButton("Average Car Cost")
        btn2.clicked.connect(self.show_avg_cost)
        layout.addWidget(btn2)

        btn3 = QPushButton("Search for a car")
        btn3.clicked.connect(self.search_car)
        layout.addWidget(btn3)

        btn4 = QPushButton("20 Random Customer Purchases")
        btn4.clicked.connect(self.show_random_customers)
        layout.addWidget(btn4)

        self.setCentralWidget(menu)

    def show_avg_cost(self):
        # Use QSqlQuery to run the average car cost query
        q = QSqlQuery(self.db)
        q.exec_("SELECT AVG(car_cost) FROM Car")
        if q.next():
            avg_cost = q.value(0)
            QMessageBox.information(self, "Average Car Cost", f"${avg_cost:.2f}")
        else:
            QMessageBox.warning(self, "Error", "Could not compute average.")

    def search_car(self):
        # Build a simple dropdown to pick an EV make
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(QLabel("Select type of make:"))
        combo = QComboBox()
        layout.addWidget(combo)

        q = QSqlQuery(self.db)
        q.exec_("SELECT DISTINCT ev_make FROM EV_arrivals ORDER BY ev_make")
        while q.next():
            combo.addItem(str(q.value(0)))

        back = QPushButton("Back to Menu")
        back.clicked.connect(self.init_menu)
        layout.addWidget(back)
        self.setCentralWidget(widget)

    def show_random_customers(self):
        # Display 20 random customers and their car purchases
        sql = (
            "SELECT * FROM ("
            "  SELECT c.first_name AS First_name, c.last_name AS Last_name,"
            "         GROUP_CONCAT(CONCAT(cs.car_make, ' ', cs.car_model) SEPARATOR ', ')"
            "         AS Cars_Bought"
            "  FROM Customers c"
            "  JOIN car_sales cs ON c.customer_id = cs.customer_id"
            "  GROUP BY c.first_name, c.last_name"
            ") AS grouped_customers"
            " ORDER BY RAND()"
            " LIMIT 20"
        )
        model = QSqlQueryModel(self)
        model.setQuery(sql, self.db)
        err = model.lastError().text()
        if err:
            QMessageBox.warning(self, "Query Error", err)
            return

        widget = QWidget()
        layout = QVBoxLayout(widget)
        view = QTableView()
        view.setModel(model)
        view.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        layout.addWidget(view)

        back = QPushButton("Back to Menu")
        back.clicked.connect(self.init_menu)
        layout.addWidget(back)
        self.setCentralWidget(widget)

    def show_table(self, tbl_name: str):
        # Display a full table view
        model = QSqlTableModel(self, self.db)
        model.setTable(tbl_name)
        model.select()

        widget = QWidget()
        layout = QVBoxLayout(widget)
        view = QTableView()
        view.setModel(model)
        view.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        layout.addWidget(view)

        back = QPushButton("Back to Menu")
        back.clicked.connect(self.init_menu)
        layout.addWidget(back)
        self.setCentralWidget(widget)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    win = OperationalApp()
    win.show()
    sys.exit(app.exec_())


SystemExit: 0

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
cursor.close()
conn.close()